## Stable Diffusion Model download with local mode testing

Stable Diffusion on HuggingFace
  - 2.1 version : https://huggingface.co/stabilityai/stable-diffusion-2-1
  - 2.1 unclip version : https://huggingface.co/stabilityai/stable-diffusion-2-1-unclip
  
  
  
### Tested version

Tested on `Python 3.9.15`

```
sagemaker: 2.146.0
transformers: 4.29.2
torch: 1.13.1
scipy: 1.8.1
safetensors: 0.3.1
diffusers: 0.16.1
```

In [ ]:
# To test Stable Diffusion as a local mode
!pip install -q diffusers transformers accelerate scipy safetensors

In [ ]:
import sagemaker
import transformers
import torch
import accelerate
print(sagemaker.__version__)
print(transformers.__version__)
print(torch.__version__)
print(accelerate.__version__)

In [ ]:
sagemaker_session = sagemaker.Session()

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path
import os

local_model_path = Path("./pretrained-models")
local_model_path.mkdir(exist_ok=True)

unclip_model_name = "stabilityai/stable-diffusion-2-1-unclip"
sd21_unclip_model_path = snapshot_download(
    repo_id=unclip_model_name,
    cache_dir=local_model_path,
)

In [ ]:
basic_model_name = "stabilityai/stable-diffusion-2-1"
sd21_basic_model_path = snapshot_download(
    repo_id=basic_model_name,
    cache_dir=local_model_path
)

In [ ]:
sd20_model_name = "stabilityai/stable-diffusion-2"
sd20_basic_model_path = snapshot_download(
    repo_id=sd20_model_name,
    cache_dir=local_model_path
)

In [ ]:
print(f"Basic Model downloaded: {sd21_basic_model_path}")
print(f"Unclip Model downloaded: {sd21_unclip_model_path}")
print(f"SD20 model downloaded: {sd20_basic_model_path}")

In [ ]:
s3_model_prefix = "llm/stable-diffusion/model"  # folder where model checkpoint will go

In [ ]:
basic_model_s3 = f"{s3_model_prefix}/sd21"
unclip_model_s3 = f"{s3_model_prefix}/sd21-unclip"
s20_model_s3 = f"{s3_model_prefix}/sd20"

In [ ]:
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

# model_path = sd21_basic_model_path
model_path = sd20_basic_model_path

# Use the DPMSolverMultistepScheduler (DPM-Solver++) scheduler here instead
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")


In [ ]:
import os
import uuid
from IPython.display import Image

def generate_image(prompt):
    image = pipe(prompt).images[0]
    local_output = Path("./test-output")
    local_output.mkdir(exist_ok=True)
    img_path = os.path.join(local_output, str(uuid.uuid4()) + ".png")
    image.save(img_path)
    display(Image(filename=img_path))

In [ ]:
prompt = "A painting which makes me smile, detail and fine art style"
generate_image(prompt)

### Upload model to s3

- If models works well on local mode, upload model files to s3
- It will be used for deployment. Directly downloading models from HF model hub can be slow and not for stable for production service, I will use S3 for model registry.

### To reduce data upload/download speed

- StableDiffusion provide two type of models (ema, nonema) for the normal checkpoint and safetensor format. To reduce data upload time and download time for deployment, just choose what you need.


In [ ]:
sd21_model_artifact = sagemaker_session.upload_data(path=sd21_basic_model_path, key_prefix=basic_model_s3)
print(sd21_model_artifact)

In [ ]:
sd21_unclip_model_artifact = sagemaker_session.upload_data(path=sd21_unclip_model_path, key_prefix=unclip_model_s3)
print(sd21_unclip_model_artifact)

In [ ]:
sd20_model_artifact = sagemaker_session.upload_data(path=sd20_basic_model_path, key_prefix=s20_model_s3)
print(sd20_model_artifact)

In [ ]:
%store sd21_basic_model_path
%store sd21_unclip_model_path
%store sd21_model_artifact
%store sd21_unclip_model_artifact

In [ ]:
%store sd20_model_artifact
%store sd20_basic_model_path